In [ ]:
from pathlib import Path
from datetime import datetime
import subprocess
import json
import copy
import numpy as np
from typing import Dict, List, Union

In [ ]:
def copy_every_n_images(images_path: Path, new_image_dir: Path, n: int = 2, offset: int = 0):
    count = 0
    new_image_dir.mkdir(exist_ok=True)
    # Copy every n image from images_path to new_image_dir and rename it to be in the format %04d.png, the copy needs to be sorted by name
    for image_path in sorted(images_path.glob("*.png"))[offset::n]:
        new_image_path = new_image_dir / f"{count:04d}.png"
        subprocess.run(["cp", str(image_path), str(new_image_path)])
        count += 1

def copy_every_n_transforms(transforms_path: Path, new_transforms_path: Path, n: int = 2, offset: int = 0):
    with open(transforms_path, "r") as f:
        transforms = json.load(f)
    
    frames = transforms["frames"]
    new_frames = [frame for i, frame in enumerate(frames) if (i + offset) % n == 0]
    new_transforms = copy.deepcopy(transforms)
    new_transforms["frames"] = new_frames

    with open(new_transforms_path, "w") as f:
        json.dump(new_transforms, f, indent=4)

In [ ]:
def load_json(path: Union[Path, str]):
    with open(path, "r") as f:
        return json.load(f)

In [ ]:
### FROM block_nerf/block_nerf.py

def transform_camera_path(camera_path_path: Path, dataparser_transform_path: Path, export_path: Path = None):
    """
    Transform a un-transformed camera path to a transformed camera path, in the respective transform's coordinate system.
    """
    camera_path = load_json(camera_path_path)
    dataparser_transform = load_json(dataparser_transform_path)

    t = np.array(dataparser_transform["transform"])
    s = dataparser_transform["scale"]

    for i, camera in enumerate(camera_path["camera_path"]):
        c2w = np.array(camera["camera_to_world"]).reshape(4, 4)
        c2w = (t @ c2w) * s
        c2w = np.vstack((c2w, np.array([0, 0, 0, 1])))
        camera["camera_to_world"] = c2w.reshape(16).tolist()

    export_path = export_path if export_path is not None else camera_path_path.parent / "camera_path_transformed.json"
    with open(export_path, "w") as f:
        json.dump(camera_path, f, indent=4)

    print("✅ Created transformed camera path at: ", export_path)
    return export_path

In [ ]:
def create_camera_path_from_transforms(transforms_path: Path, camera_path_path: Path, dataparser_transforms_path: Path, fps: int = 24):
    with open(transforms_path, "r") as f:
        transforms = json.load(f)
    
    frames = transforms["frames"]
    flattened_frames = [np.array(frame["transform_matrix"]).flatten().tolist() for frame in frames]

    new_camera_path = {
        "keyframes": [],
        "camera_type": "perspective",
        "render_height": 1080,
        "render_width": 1920,
        "camera_path": [], # "camera_to_world"-dict with 16 double values in a 1D list.
        "fps": fps,
        "seconds": len(frames) / fps,
        "smoothness_value": 0.46249999999999997,
        "is_cycle": "true"
    }

    for c2w in flattened_frames:
        new_camera_path["camera_path"].append({
            "camera_to_world": c2w,
            "fov": 90,
            "aspect": 1.6678200692041523 # TODO: Should I get this somewhere else?
    })
    
    with open(camera_path_path, "w") as f:
        json.dump(new_camera_path, f, indent=4)
    
    # Scale and translate the camera path to the trained NeRF's coordinate system.
    # with open(dataparser_transforms_path, "r") as f:
    #     dataparser_transforms = json.load(f)
    
    transform_camera_path(camera_path_path, dataparser_transforms_path, camera_path_path.parent / camera_path_path.name.replace(".json", "_scaled.json"))

In [ ]:
def get_timestamp():
    return datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
def create_video_from_images(images_path: Path, export_dir: Path, fps: int = 24):
    export_dir.mkdir(exist_ok=True)
    output_name = export_dir / f"output-{get_timestamp()}.mp4"
    cmd = f"ffmpeg -framerate {fps} -i {images_path}/%04d.png -c:v libx264 -r {fps} -pix_fmt yuv420p {output_name}"
    print(cmd)
    subprocess.run(cmd, shell=True, check=True)


In [ ]:
transforms_path = Path("./transforms.json")
new_transforms_path = Path("./new_transforms.json")
images_path = Path("./images")
new_image_dir = Path("./new_images")
camera_path_path = Path(f"camera_path-{get_timestamp()}.json")
dataparser_transforms_path = Path("./dataparser_transforms.json")
render_dir = Path("./renders")

# Right camera in a rig with 2 cameras
n = 2
offset = 1 
fps = 24

copy_every_n_transforms(transforms_path, new_transforms_path, n=n, offset=offset)
copy_every_n_images(images_path, new_image_dir, n=n, offset=offset)
create_video_from_images(new_image_dir, export_dir=render_dir, fps=fps)
create_camera_path_from_transforms(transforms_path=new_transforms_path, camera_path_path=camera_path_path, dataparser_transforms_path=dataparser_transforms_path, fps=fps)

In [ ]:
exp_path = Path("../data/images/side_by_side_test/0")
model_path = exp_path / "side_by_side_test-0/nerfacto/2023-04-22_160821"

transforms_path = exp_path / "transforms.json"
new_transforms_path = exp_path / "new_transforms.json"
images_path = exp_path / "images"
new_image_dir = exp_path / "new_images"
camera_path_path = exp_path / f"camera_path-{get_timestamp()}.json"
dataparser_transforms_path = model_path / "dataparser_transforms.json"
render_dir = exp_path / "renders"

# Right camera in a rig with 2 cameras
n = 2
offset = 1 
fps = 24

copy_every_n_transforms(transforms_path, new_transforms_path, n=n, offset=offset)
copy_every_n_images(images_path, new_image_dir, n=n, offset=offset)
create_video_from_images(new_image_dir, export_dir=render_dir, fps=fps)
create_camera_path_from_transforms(transforms_path=new_transforms_path, camera_path_path=camera_path_path, dataparser_transforms_path=dataparser_transforms_path, fps=fps)

In [ ]:
def create_side_by_side_video(video_paths: List[Path], export_path: Path):
    # Rescale all input-videos to 1080p
    rescaled_paths = []
    for i, video_path in enumerate(video_paths):
        rescaled_video_path = video_path.parent / f"rescaled-{video_path.name}"
        cmd = f"ffmpeg -n -i {video_path} -vf \"scale=w=960:h=540:force_original_aspect_ratio=1,pad=960:540:(ow-iw)/2:(oh-ih)/2\" -c:v libx264 {rescaled_video_path}"
        print(cmd)
        subprocess.run(cmd, shell=True, check=True)
        rescaled_paths.append(rescaled_video_path)
    print("✅ Rescaled videos to 1080p")

    # Create side-by-side videoP
    cmd = f"ffmpeg -n {' '.join([f'-i {video_path}' for video_path in rescaled_paths])} -filter_complex hstack=inputs={len(rescaled_paths)} {export_path}"
    print(cmd)
    subprocess.run(cmd, shell=True, check=True)

In [ ]:
model_render_path = Path("../data/images/side_by_side_test/0/renders/model_render.mp4")
input_render_path = Path("../data/images/side_by_side_test/0/renders/output-2023-04-22_16-20-16.mp4")

create_side_by_side_video([model_render_path, input_render_path], export_path=Path("../data/images/side_by_side_test/0/renders/side_by_side.mp4"))